# Wildfire Hazard Potential for the United States (270-m), version 2023 (4th Edition)

Dataset source: https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_fileindex_RDS-2015-0047-4.html

Documentation: https://firelab.org/sites/default/files/2024-02/whp2023_cls_conus_metadata.pdf

https://data-usfs.hub.arcgis.com/datasets/usfs::wildfire-hazard-potential-version-2023-classified-image-service/explore?location=35.134247%2C-116.251988%2C6.95

https://www.fs.usda.gov/rds/archive/catalog/RDS-2015-0047-4


## Data dictionary

https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_metadata_RDS-2015-0047-4.html

In [17]:
pip install rioxarray

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import os
import random

from glob import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rioxarray
import xarray as xr

from rasterio.enums import Resampling
from rasterio.errors import WindowError
from rasterio.features import geometry_mask
from rasterio.mask import mask
from rasterio.windows import Window

from shapely.geometry import Point, box

In [2]:
whp_filename = '../../WData/whp2023_GeoTIF/whp2023_cnt_conus.tif'

with rasterio.open(whp_filename) as src2:
    bounds = src2.bounds
    bbox = box(*bounds)
    geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=src2.crs)
    geo_json = [json.loads(geo.to_json())['features'][0]['geometry']]

print(len(geo_json))


1


In [4]:
# Path to the folder containing .tif files
folder_path = "../../sim2real_layouts"
os.makedirs("risk_layouts/", exist_ok=True)

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))
print(tif_files)

layout_list = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[2].split('_E')[0]
    print(f"Processing {id}...")

    output_path = os.path.join("risk_layouts", f"whpi2023_cnt_{id}.tif")
    try:
        with rasterio.open(tif_file) as src1:
            # print the resolution of the raster in meters
            print(src1.res)
            out_image, out_transform = mask(src1, geo_json, crop=True)
            out_meta = src1.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Save cropped image
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
    except (ValueError, WindowError) as e:
        print(f"Error masking raster: {e}")
        
    print(f"Saved cropped intersection as {output_path}")

['../../sim2real_layouts/0025_02019_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0049_01289_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0065_03061_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0089_00984_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0057_03186_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0255_02103_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0023_00995_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0012_02094_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0020_00970_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0067_03550_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0111_03612_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0092_03189_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0016_03070_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0068_04211_Existing_Vegetation_Cover.tif', '../../sim2real_layouts/0246_00984_Existing_Vegetation_Cover.

In [6]:
tif_files = glob(os.path.join("risk_layouts", "*.tif"))
for tif_file in tif_files:
    # print(f"Processing {tif_file}...")
    id = "_".join(tif_file.split('/')[1].split('_E')[0].split('_')[-3:]).split('.')[0]
    print(id)
    with rasterio.open(tif_file) as src:
        wfpi_data = src.read(1)
        print(wfpi_data.shape)
        # print resolution of the raster in meters
        print(src.res)
    plt.imshow(wfpi_data, cmap='hot', vmin=0, vmax=500)
    plt.colorbar(label="WFPI Value")
    plt.title("Wind-enhanced Fire Potential Index (WFPI)")
    plt.show()
    # print the corresponding layout png
    layout_png_file = os.path.join("../WideDataset", f"{id}/satellite_image.png")
    layout_png = plt.imread(layout_png_file)
    plt.imshow(layout_png)
    plt.show()



cnt_sim2real_layouts
(1046, 846)
(30.0, 30.0)


FileNotFoundError: [Errno 2] No such file or directory: '../WideDataset/cnt_sim2real_layouts/satellite_image.png'

In [6]:
def convert_tif_to_npy(input_folder, output_folder):
    """
    Convert all .tif or .tiff files in a folder to .npy format.

    Args:
        input_folder (str): Path to the directory containing .tif/.tiff files.
        output_folder (str): Path to the directory where .npy files will be saved.
        max_files (int, optional): Maximum number of .tif files to process. If None, process all files.

    Yields:
        str: Path to each .npy file created.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file in os.listdir(input_folder):
        if file.lower().endswith('.tif') or file.lower().endswith('.tiff'):
            tif_path = os.path.join(input_folder, file)
            with rasterio.open(tif_path) as src:
                data = src.read(1)  # Read the first band
                # duplicate the data to go from shape (N,M) to shape (100,N,M)
                data = np.tile(data, (1, 1, 1))
                print(data.shape)

            npy_filename = os.path.splitext(file)[0] + '.npy'
            np.save(os.path.join(output_folder, npy_filename), data)
            print(f"Converted {file} to {npy_filename}")

In [7]:
# convert tif files to npy files
convert_tif_to_npy("./risk_layouts", "./risk_layouts_npy")

(1, 919, 1041)
Converted whpi2023_cnt_0062_03187.tif to whpi2023_cnt_0062_03187.npy
(1, 783, 770)
Converted whpi2023_cnt_0243_02722.tif to whpi2023_cnt_0243_02722.npy
(1, 819, 930)
Converted whpi2023_cnt_0057_03186.tif to whpi2023_cnt_0057_03186.npy
(1, 551, 607)
Converted whpi2023_cnt_0041_02386.tif to whpi2023_cnt_0041_02386.npy
(1, 770, 819)
Converted whpi2023_cnt_0247_03453.tif to whpi2023_cnt_0247_03453.npy
(1, 655, 681)
Converted whpi2023_cnt_0259_02663.tif to whpi2023_cnt_0259_02663.npy
(1, 1046, 846)
Converted whpi2023_cnt_0103_01810.tif to whpi2023_cnt_0103_01810.npy
(1, 945, 1163)
Converted whpi2023_cnt_0098_01784.tif to whpi2023_cnt_0098_01784.npy
(1, 815, 821)
Converted whpi2023_cnt_0064_02717.tif to whpi2023_cnt_0064_02717.npy
(1, 256, 506)
Converted whpi2023_cnt_0344_03155.tif to whpi2023_cnt_0344_03155.npy
(1, 1046, 846)
Converted whpi2023_cnt_0102_01733.tif to whpi2023_cnt_0102_01733.npy
(1, 1323, 1484)
Converted whpi2023_cnt_0048_01141.tif to whpi2023_cnt_0048_01141.np

In [14]:
# move the .npy files from the risk_layouts folder to the appropriate folder
import shutil
import os
from glob import glob

# Path to the folder containing .tif files
folder_path = "../sim2real_layouts"

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))

layout_list = []
missing_layouts = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[2].split('_E')[0]
    print(id)
    rskfile_path = os.path.join("risk_layouts_npy", f"whpi2023_cnt_{id}.npy")
    if os.path.exists(rskfile_path):
        #print("copying", rskfile_path, "to", "./WideDataset", f"{id}/static_risk.npy")
        try:
            shutil.copy(rskfile_path, os.path.join("./WideDataset", f"{id}/static_risk.npy"))
        except Exception as e:
            print(f"Error copying {rskfile_path} : {e}")
    else:
        missing_layouts.append(id)
print(len(missing_layouts), "layouts skipped, copied", len(tif_files) - len(missing_layouts), "layouts")


0025_02019
Error copying risk_layouts_npy/whpi2023_cnt_0025_02019.npy : [Errno 2] No such file or directory: './WideDataset/0025_02019/static_risk.npy'
0049_01289
Error copying risk_layouts_npy/whpi2023_cnt_0049_01289.npy : [Errno 2] No such file or directory: './WideDataset/0049_01289/static_risk.npy'
0065_03061
Error copying risk_layouts_npy/whpi2023_cnt_0065_03061.npy : [Errno 2] No such file or directory: './WideDataset/0065_03061/static_risk.npy'
0089_00984
Error copying risk_layouts_npy/whpi2023_cnt_0089_00984.npy : [Errno 2] No such file or directory: './WideDataset/0089_00984/static_risk.npy'
0057_03186
Error copying risk_layouts_npy/whpi2023_cnt_0057_03186.npy : [Errno 2] No such file or directory: './WideDataset/0057_03186/static_risk.npy'
0255_02103
0023_00995
Error copying risk_layouts_npy/whpi2023_cnt_0023_00995.npy : [Errno 2] No such file or directory: './WideDataset/0023_00995/static_risk.npy'
0012_02094
Error copying risk_layouts_npy/whpi2023_cnt_0012_02094.npy : [Errn

In [15]:
print(missing_layouts)

['0255_02103', '0258_02858', '0257_02175', '0253_03246', '0113_03495', '0254_02361', '0114_02292', '0256_02752']
